# generate / update a DataDiVR backend project 

In [ ]:
# functions to mimick uploading process (generate folders, files on the backend)

# PROCESS STEPS 
# do something with a Graph 
# generate JSON files per layout for the Graph (temporary solution?)
# use those JSON to generate folders, files like textures, nodes.json, links.json etc.. 

In [ ]:
# some functions : 

import networkx as nx
import random
import json

def generate_random_rgba():
    red = random.randint(0, 255)
    green = random.randint(0, 255)
    blue = random.randint(0, 255)
    alpha = random.randint(0, 255)  # Random alpha value between 0 and 1
    
    rgba_color = (red, green, blue, alpha)
    return rgba_color


def rgba_to_hex8(rgba_color):
    # Convert values to hexadecimal and format them
    hex_red = format(rgba_color[0], '02X')
    hex_green = format(rgba_color[1], '02X')
    hex_blue = format(rgba_color[2], '02X')
    hex_alpha = format(rgba_color[3], '02X')
    
    # Combine values into an 8-character hexadecimal color code
    color_hex = f"#{hex_red}{hex_green}{hex_blue}{hex_alpha}"
    
    return color_hex


In [ ]:
# generate a graph 

G = nx.karate_club_graph()
print("Number of nodes: ", len(G.nodes()))
print("Number of Links: ", len(G.edges()))

nx.draw(G)

In [ ]:
# generate graph layouts i.e. node positions 
posG3D_1_pre = nx.spring_layout(G, dim=3, k=0.1, iterations=100)
posG3D_1 = {key: value.tolist() for key, value in posG3D_1_pre.items()}

posG3D_2_pre = nx.spring_layout(G, dim=3, k=0.1, iterations=200)
posG3D_2= {key: value.tolist() for key, value in posG3D_2_pre.items()}

posG3D_3_pre =nx.spring_layout(G, dim=3, k=0.1, iterations=500)
posG3D_3 = {key: value.tolist() for key, value in posG3D_3_pre.items()}

In [ ]:
# hex color values 
d_nodecolors_hex = dict(zip(G.nodes(),['#FF2300']*len(G.nodes())))
l_linkcolors_hex = '#ff0000'

# rgba color values
d_nodecolors_rgba = dict(zip(G.nodes(),[(255,35,0,120)]*len(G.nodes())))
l_linkcolors_rgba = (0,255,0,100)

# hex8 color values
d_nodecolors_hex8 = dict(zip(G.nodes(),['#0000ffaa']*len(G.nodes())))
l_linkcolors_hex8 = '#0080ffaa'

### JSON 

In [ ]:
import json

In [ ]:
# ===============================================
# NODE ANNOTATION - a list of strings
# ===============================================

# new annotation format: 
""" 
	"nodes": [
		{
			"pos": [
				20.157665194728846,
				19.08545649237665,
				10.958288870722058
			],
			"cluster": "cluster: 0",
			"nodecolor": "#1f77b4",
			"annotation": {
				"Movies": [
					"The Dark Knight",
					"The Dark Knight Rises"
				],
				"Genres": [
					"Drama",
					"Thriller",
					"Action",
					"Crime"
				]
			},
			"id": 0,
			"name": "Aaron Eckhart"
		}, """

# --------------------------------------------------------------------
#
# TO DO : INCORPORATE NEW ANNOTATIONS FORMAT INTO THE GRAPH
#
# --------------------------------------------------------------------

l_annotations_json = []
for g in G.nodes():
    sublist = ["Node: "+str(g)]
    l_annotations_json.append(sublist)
        
d_annotations = dict(zip(G.nodes(), l_annotations_json))
nx.set_node_attributes(G, d_annotations, name="annotation")





# ===============================================
# GRAPH NAME AND DESCRIPTION - a string each
# ===============================================

G.graph['graphtitle']  = "Zachary Karateclub Graph"
G.graph['graphdesc'] = "A toy graph for testing purposes. Number of nodes: "+str(len(G.nodes()))+", Links: "+ str(len(G.edges()))+"."

# ===============================================
# NODE COLORS - a dict with keys = node id as in G.nodes and values = color as hex or rgba
# LINK COLORS - a list of hex or rgba colors per edge as in G.edges or one color for all edges
# ===============================================
#                     a n d 
# ===============================================
# NODE POSITIONS - a dict with keys = G.nodes and values = coordinates (x,y) or (x,y,z)
# ===============================================

# first Layout - rgba
G_rgba = G.copy()
G_rgba.name = '01-graph-rgbacolors'
nx.set_node_attributes(G_rgba, d_nodecolors_rgba, name="nodecolor")
nx.set_edge_attributes(G_rgba, l_linkcolors_rgba, name="linkcolor")
nx.set_node_attributes(G_rgba, posG3D_1, name="pos")

# second Layout - hex
G_hex = G.copy()
G_hex.name = '02-graph-hexcolors'
nx.set_node_attributes(G_hex, d_nodecolors_hex, name="nodecolor")
nx.set_edge_attributes(G_hex, l_linkcolors_hex, name="linkcolor")
nx.set_node_attributes(G_hex, posG3D_2, name="pos")

# third Layout - hex8
G_hex8 = G.copy()
G_hex8.name = '03-graph-hex8colors'
nx.set_node_attributes(G_hex8, d_nodecolors_hex8, name="nodecolor")
nx.set_edge_attributes(G_hex8, l_linkcolors_hex8, name="linkcolor")
nx.set_node_attributes(G_hex8, posG3D_3, name="pos")


# fourth layout - cluster key next level communities
# artifical groups of graph for cluster colors 
G_clusters = G.copy()
G_clusters.name = '04-graph-clustershex8colors'


clustername_1 = 'group 1'
clustername_2 = 'group 2'
clustername_3 = 'group 3'

# nodes into groups
for g in G_clusters.nodes():
    if g < len(G_clusters.nodes()) / 3:
        G_clusters.nodes[g]['cluster'] = clustername_1
    elif g < 2 * len(G_clusters.nodes()) / 3:
        G_clusters.nodes[g]['cluster'] = clustername_2
    else:
        G_clusters.nodes[g]['cluster'] = clustername_3

# node colors 
d_nodecolors_clusters = {}
nodes_group1 = []
nodes_group2 = []
nodes_group3 = []
for n in G_clusters.nodes(): 
    if G_clusters.nodes[n]['cluster'] == clustername_1:
        d_nodecolors_clusters[n] = '#0000ffaa'
        nodes_group1.append(n)
    elif G_clusters.nodes[n]['cluster'] == clustername_2:
        d_nodecolors_clusters[n] = '#00ff00aa'
        nodes_group2.append(n)
    elif G_clusters.nodes[n]['cluster'] == clustername_3:
        d_nodecolors_clusters[n] = '#ff0000aa'
        nodes_group3.append(n)

# link colors
d_linkcolors_clusters = {}
for edge in G_clusters.edges():
    if edge[0] in nodes_group1 and edge[1] in nodes_group1:
        d_linkcolors_clusters[edge] = '#0000ffaa'
       
    elif edge[0] in nodes_group2 and edge[1] in nodes_group2:
        d_linkcolors_clusters[edge] = '#00ff00aa'
       
    elif edge[0] in nodes_group3 and edge[1] in nodes_group3:
        d_linkcolors_clusters[edge] = '#ff0000aa'
       
    else:
        d_linkcolors_clusters[edge] = '#B1B1B150'

l_linkcolors_clusters = list(d_linkcolors_clusters.values())

nx.set_node_attributes(G_clusters, d_nodecolors_clusters, name="nodecolor")
nx.set_node_attributes(G_clusters, posG3D_2, name="pos") # reuse the second layout
nx.set_edge_attributes(G_clusters, {edge: color for edge, color in zip(G_clusters.edges(), l_linkcolors_clusters)}, "linkcolor")


+ the above cells are unfinished and the graph json creation process should be adapted to the new json format

# process a pre-defined json file: 

+ make json or dict? 
+ pull backend from github
+ open a notebook where you process the data you want to explore in the VR and save it as a json file
+ locate the notebook in the "DataDiVR_webapp" folder called "_GenerateProject.ipynb"
+ in this notebook : from DataDiVR_webapp import "the upload functions"
+ in this notebook : call the uploading function (as in cells below) and fill in your data object 
+ it automatically generates a folder with all files, subfolders in your backend path

In [3]:
import json
import glob

def transform_annotation(annotation_list):
    """
    Transforms a list containing a single string of annotations into a dictionary.
    The string is expected to contain annotations separated by '/',
    with each annotation in the format 'key: value'.
    Example input: ['ID: 1/degree: 10']
    Example output: {'ID': '1', 'degree': '10'}
    """
    annotation_dict = {}
    if annotation_list:  # Check if the list is not empty
        annotations = annotation_list[0].split('/')
        for annotation in annotations:
            key, value = annotation.split(': ', 1)  # Split on the first occurrence of ': '
            annotation_dict[key.strip()] = value.strip()
    return annotation_dict

def merge_json_files(file_paths):
    all_nodes = []
    all_links = []
    layouts = []

    for file_path in file_paths:
        with open(file_path, 'r') as file:
            data = json.load(file)

            # Process nodes for global and layout-specific lists
            for node in data["nodes"]:
                if "annotation" in node and isinstance(node["annotation"], list):
                    node["annotation"] = transform_annotation(node["annotation"])
                all_nodes.append({'id': node['id'], 'annotation': node.get('annotation', {})})

            # Process links for global list, now with separate source and target
            for link in data["links"]:
                all_links.append({
                    #'w': link['weight'],
                    'source': link['source'],
                    'target': link['target']
                })

            # Prepare layout-specific nodes and links
            layout_nodes = [{'nodecolor': node.get('nodecolor', ''),
                             'pos': node.get('pos', []),
                             'id': node['id']} for node in data["nodes"]]
            layout_links = [{'linkcolor': link.get('linkcolor', ''),
                             'source': link['source'],
                             'target': link['target']} for link in data["links"]]

            layouts.append({'nodes': layout_nodes, 'links': layout_links})

    # Use the structure of the first file as the base for the merged structure
    merged_structure = {}
    if file_paths:
        with open(file_paths[0], 'r') as file:
            base_structure = json.load(file)
            # Keep only the top-level keys except "nodes" and "links"
            for key in ['directed', 'multigraph', 'graph']:
                merged_structure[key] = base_structure.get(key)

    # Add the global nodes and links, and the layouts to the merged structure
    merged_structure['nodes'] = all_nodes
    merged_structure['links'] = all_links
    merged_structure['layouts'] = layouts

    return merged_structure

In [4]:
folder = "temp_upload_data/barbell/"
file_paths = glob.glob(folder + '*.json')  # Update this path
merged_json = merge_json_files(file_paths)

# Save the merged JSON to a file
with open(folder + 'merged.json', 'w') as outfile:
    json.dump(merged_json, outfile, indent=4)

print("Merged JSON saved as 'merged.json'.")

Merged JSON saved as 'merged.json'.


## Produce Files for a DataDiVR Project 

In [1]:
import json 
temp_folder = "temp_upload_data/"
file = "merged.json" 

with open (temp_folder+file, "r") as f:
    data = json.load(f)

In [2]:
from uploaderGraph import *
from uploader import *

upload_filesJSON(data)

C_DEBUG: upload via Notebook function.
Successfully created the directory static/projects/Sphere-100nodes 
hight is 128
hight is 128
hight is 128
hight is 128
C_DEBUG: project does not contain labels/clusters.
hight is 128
hight is 128
hight is 128
hight is 128
image hight = 64
image hight = 64
image hight = 64
image hight = 64
['Barbellgraph', 'JSON_autocore', 'JSON_barbellgraph', 'JSON_Zachary', 'Sphere-100nodes']


'<a style="color:green;">SUCCESS </a>LayoutID0 Node Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID1 Node Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID2 Node Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID3 Node Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID0 Node Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID1 Node Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID2 Node Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID3 Node Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID0 Link Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID1 Link Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID2 Link Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID3 Link Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID0_links Link Textures Created<br><a style="color:green;">SUCCESS </a>LayoutID1_links Link Textures Crea